In [4]:
from xml.dom import minidom
import glob, os
from collections import Counter
import pandas as pd
import re

In [5]:
# buscando categorias do liwc
with open('data/liwc.dic') as f:
    linhas = f.readlines()
    
liwcCats = []
for cont, linha in enumerate(linhas):
    linha = linha.strip("\n")
    if linha == "%":
        if cont > 2:
            break
        else:
            continue
    it = linha.split("\t")[1]
    #print(it)
    liwcCats.append(it)
liwcCats.sort()
print(liwcCats)

['achieve', 'adverb', 'affect', 'anger', 'anx', 'article', 'assent', 'auxverb', 'bio', 'body', 'cause', 'certain', 'cogmech', 'conj', 'death', 'discrep', 'excl', 'family', 'feel', 'filler', 'friend', 'funct', 'future', 'health', 'hear', 'home', 'humans', 'i', 'incl', 'ingest', 'inhib', 'insight', 'ipron', 'leisure', 'money', 'motion', 'negate', 'negemo', 'nonfl', 'number', 'past', 'percept', 'posemo', 'ppron', 'preps', 'present', 'pronoun', 'quant', 'relativ', 'relig', 'sad', 'see', 'sexual', 'shehe', 'social', 'space', 'swear', 'tentat', 'they', 'time', 'verb', 'we', 'work', 'you']


In [41]:
# buscando categorias deprel e udeprel
deprels = []
udeprels = []

for arquivo in sorted(glob.glob("data/BDCamões_DependencyBank/*.tsv")):
    #print(arquivo)
    df = pd.read_csv(arquivo, sep='\t')
    for x in df['deprel']:
        if (x not in deprels and not x.isnumeric()): deprels.append(x)
    for x in df['udeprel']:
        if (x not in udeprels and not x.isnumeric()): udeprels.append(x)            
    #break
deprels.sort()
udeprels.sort()

/dados/usuarios/mateus/projetos/camões/camoes/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/dados/usuarios/mateus/projetos/camões/camoes/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [42]:
print(deprels)

['ADV', 'C', 'C-ARG1', 'C-ARG2', 'C-ARG3', 'CARD', 'CONJ', 'COORD', 'D1', 'DEP', 'DO', 'DO-ARG1', 'DO-ARG2', 'DO-ARG21', 'DO-ARG3', 'DO-EXT', 'IO-ARG2', 'IO-ARG3', 'M', 'M-ADV', 'M-ARG1', 'M-CAU', 'M-DIR', 'M-EXT', 'M-LOC', 'M-M', 'M-MNR', 'M-NULL', 'M-PNC', 'M-POV', 'M-PRED', 'M-TMP', 'N', "N'", 'OBL-ADV', 'OBL-ARG1', 'OBL-ARG2', 'OBL-ARG3', 'P', 'PERCENT', 'POSS', 'PRD', 'PRD-ARG2', 'PUNCT', 'QNT', 'ROOT', 'S', 'SJ', 'SJ-ARG1', 'SJ-ARG11', 'SJ-ARG1cp', 'SJ-ARG2', 'SJ-ARG2ac', 'SJ-ARG2cp', 'SJ-ARGA', 'SJ-NULL', 'SP', 'SP-ARG1', 'SP-ARG2', 'SP-M', 'SP-PRED', 'V']


In [43]:
print(udeprels)

['ADVCL', 'ADVMOD', 'AMOD', 'APPOS', 'AUX', 'CASE', 'CC', 'CCOMP', 'COMP', 'CONJ', 'COORD', 'COP', 'CSUBJ', 'CSUBJPASS', 'DEP', 'DET', 'DISCOURSE', 'DOBJ', 'IOBJ', 'MARK', 'MOD', 'MWE', 'NSUBJ', 'NSUBJPASS', 'NUM', 'NUMBER', 'PARATAXIS', 'POBJ', 'POSS', 'PRECONJ', 'PREDET', 'PUNCT', 'ROOT', 'S', 'SJ', 'XCOMP']


## Constructing dataset for machine learnning

In [67]:
fdep = open("data/camoes_dep.csv", "w")
fdep.write("id;titulo;autor;ano;genero;" + ";".join(deprels) + "\n"  )

fudep = open("data/camoes_udep.csv", "w")
fudep.write("id;titulo;autor;ano;genero;" + ";".join(udeprels) + "\n"  )


id = 1
for arquivo in sorted(glob.glob("data/BDCamões_DependencyBank/*.tsv")):
    #print(arquivo)
    nomeArquivo = arquivo.split('/')[2]
    arq = nomeArquivo.split('.')[0]
    mydoc = minidom.parse("data/BDCamões/"+arq+".xml")
    #print(arq)

    items = mydoc.getElementsByTagName('title')
    title = items[0].firstChild.data

    items = mydoc.getElementsByTagName('author')
    author = items[0].firstChild.data

    items = mydoc.getElementsByTagName('other')
    other = items[0].firstChild.data
    
    for it in other.split("\n"):
        if "YY" in it:
            ano = it.split(":")[1].strip()
        if "GG" in it:
            genero = it.split(":")[1].strip()

    items = mydoc.getElementsByTagName('text')
    texto = items[0].firstChild.data
    #print(texto)
           
    # opening csv
    auxDepRels = []
    df = pd.read_csv(arquivo, sep='\t')

    # couting dep rels
    auxDepRels = [x for x in df['deprel'] if not x.isnumeric()]
    contDepRels = Counter(auxDepRels).most_common()

    # couting udep rels
    auxUDepRels = [x for x in df['udeprel'] if not x.isnumeric()]
    contUDepRels = Counter(auxUDepRels).most_common()
    
    
    # total de tokens na obra
    num_lines = sum(1 for line in open(arquivo))-1
    
    # dep file
    aux = str(id)+";"+title +";"+author+";"+ano+";"+genero
    for cat in deprels:
        sent = [x[1] for x in contDepRels if x[0] == cat]
        if sent:
            valor=round(sent[0]*10000/num_lines)
        else:
            valor=0
        aux = aux + ";"+str(valor)
    fdep.write(aux+"\n")

    # udep file
    aux = str(id)+";"+title +";"+author+";"+ano+";"+genero
    for cat in udeprels:
        sent = [x[1] for x in contUDepRels if x[0] == cat]
        if sent:
            valor=round(sent[0]*10000/num_lines)
        else:
            valor=0
        aux = aux + ";"+str(valor)
    fudep.write(aux+"\n")
    
    
    #if id > 3: break
    id = id + 1
fdep.close()
fudep.close()

In [68]:
!wc -l data/camoes_dep.csv

196 data/camoes_dep.csv


In [69]:
!head data/camoes_dep.csv

id;titulo;autor;ano;genero;ADV;C;C-ARG1;C-ARG2;C-ARG3;CARD;CONJ;COORD;D1;DEP;DO;DO-ARG1;DO-ARG2;DO-ARG21;DO-ARG3;DO-EXT;IO-ARG2;IO-ARG3;M;M-ADV;M-ARG1;M-CAU;M-DIR;M-EXT;M-LOC;M-M;M-MNR;M-NULL;M-PNC;M-POV;M-PRED;M-TMP;N;N';OBL-ADV;OBL-ARG1;OBL-ARG2;OBL-ARG3;P;PERCENT;POSS;PRD;PRD-ARG2;PUNCT;QNT;ROOT;S;SJ;SJ-ARG1;SJ-ARG11;SJ-ARG1cp;SJ-ARG2;SJ-ARG2ac;SJ-ARG2cp;SJ-ARGA;SJ-NULL;SP;SP-ARG1;SP-ARG2;SP-M;SP-PRED;V
1;A virtude laureada;Manoel Maria de Barbosa du Bocage;1805;Drama;27;1077;45;129;1;1;455;476;0;1360;52;7;530;1;0;0;1;65;11;355;8;20;8;57;156;57;33;0;6;5;1141;131;502;2;1;124;125;22;8;0;0;77;26;363;0;336;8;0;499;1;25;20;0;5;0;0;1158;69;0;0;1;0
2;Os Pobres;Raul Brandão;1906;Novel;23;1004;80;145;2;1;540;553;0;994;94;8;515;0;0;0;2;71;16;352;0;24;1;85;120;41;41;0;12;9;659;159;55;1;0;141;111;14;16;0;0;170;93;780;1;551;7;0;530;2;37;33;0;9;0;0;1246;54;0;0;1;0
3;Raios de extincta luz;Antero de Quental;1892;Poetry;30;1155;72;135;2;1;467;480;0;1036;54;6;446;0;0;0;0;59;16;327;0;30;1;94;155;59;40

In [70]:
!tail data/camoes_dep.csv

186;TENTAR NÃO MORRER;Gonçalo M. Tavares;2005;Tale;67;1087;107;81;0;0;362;376;0;443;188;27;416;0;0;0;0;40;67;456;0;0;0;67;107;40;67;0;27;13;295;228;0;0;0;121;242;13;13;0;0;255;27;1007;0;953;13;0;510;0;27;0;0;0;0;0;1221;107;0;0;0;0
187;CONFIDÊNCIA BARREIRENSE;Teresa Veiga;2000;Tale;27;1737;105;239;0;10;492;471;0;381;98;11;546;2;0;0;0;96;11;464;0;32;1;114;164;45;58;0;43;11;605;216;63;0;0;354;239;23;46;0;3;178;69;317;2;312;5;0;421;2;60;30;0;10;0;0;1482;97;0;0;0;0
188;O ROMANCE PERDIDO;Fernando Venâncio;2004;Tale;42;1214;77;162;2;0;405;432;0;1041;92;30;577;0;0;0;7;77;2;437;0;25;2;115;172;55;52;0;10;10;544;240;20;0;0;227;135;20;25;0;0;182;90;524;0;547;7;0;425;0;55;32;0;10;0;0;1276;70;0;0;0;0
189;Auto da Barca do Inferno;Gil Vicente;1517;Drama;17;888;95;189;0;4;407;428;0;639;27;37;541;0;0;0;0;65;9;285;0;33;0;67;135;46;56;0;1;4;680;114;366;0;0;141;148;10;9;0;0;164;80;811;0;752;11;0;666;6;58;38;0;13;0;0;969;58;0;0;0;0
190;Auto da Alma;Gil Vicente;1518;Drama;11;924;77;181;2;6;507;518;0;1160;32;

In [71]:
!wc -l data/camoes_udep.csv

196 data/camoes_udep.csv


In [72]:
!head data/camoes_udep.csv

id;titulo;autor;ano;genero;ADVCL;ADVMOD;AMOD;APPOS;AUX;CASE;CC;CCOMP;COMP;CONJ;COORD;COP;CSUBJ;CSUBJPASS;DEP;DET;DISCOURSE;DOBJ;IOBJ;MARK;MOD;MWE;NSUBJ;NSUBJPASS;NUM;NUMBER;PARATAXIS;POBJ;POSS;PRECONJ;PREDET;PUNCT;ROOT;S;SJ;XCOMP
1;A virtude laureada;Manoel Maria de Barbosa du Bocage;1805;Drama;21;259;569;8;3;591;316;27;0;18;0;75;1;0;3920;788;0;435;54;61;620;395;382;1;7;10;204;174;92;1;0;300;227;0;0;26
2;Os Pobres;Raul Brandão;1906;Novel;35;411;525;2;19;672;483;76;1;26;0;246;2;0;1933;1144;0;609;61;115;527;105;548;1;7;0;235;236;74;2;1;740;506;0;0;61
3;Raios de extincta luz;Antero de Quental;1892;Poetry;41;351;548;5;7;749;370;50;0;30;0;184;1;0;2566;1049;0;457;50;107;554;296;450;1;12;18;172;315;80;4;0;544;368;0;0;45
4;Dispersão;Mário Sá-Carneiro;1914;Petry;49;418;416;15;3;610;433;67;0;17;0;218;6;0;2264;683;0;575;78;110;546;221;491;0;0;0;259;209;81;6;0;727;546;0;0;46
5;Cantos Sagrados;Manuel de Arriaga;1899;Poetry;24;272;643;5;10;870;359;53;0;32;0;127;1;0;2903;1161;0;404;34;88;644;225;384;

In [73]:
!tail data/camoes_udep.csv

186;TENTAR NÃO MORRER;Gonçalo M. Tavares;2005;Tale;13;523;336;0;54;832;309;40;0;13;0;282;0;0;1329;1087;0;604;27;67;336;161;443;0;27;0;188;322;107;0;0;980;913;0;0;81
187;CONFIDÊNCIA BARREIRENSE;Teresa Veiga;2000;Tale;45;502;586;8;31;1246;449;195;1;35;0;231;8;0;1247;1363;0;667;87;204;633;169;467;6;30;0;73;559;146;3;2;310;300;0;0;93
188;O ROMANCE PERDIDO;Fernando Venâncio;2004;Tale;40;609;475;2;30;834;355;75;2;30;0;252;5;2;1968;1144;0;654;80;147;475;90;490;2;40;0;200;320;90;0;0;509;504;0;0;47
189;Auto da Barca do Inferno;Gil Vicente;1517;Drama;28;430;458;9;16;582;350;67;14;47;0;232;0;0;1443;769;0;611;63;104;512;391;713;0;14;141;169;245;67;0;0;801;727;0;1;63
190;Auto da Alma;Gil Vicente;1518;Drama;13;341;549;8;15;532;385;66;2;34;0;209;2;0;3098;650;0;522;41;90;430;279;507;2;6;187;175;160;87;0;0;466;419;0;0;38
191;Auto da Índia;Gil Vicente;1509;Drama;27;431;497;5;5;367;314;53;0;27;0;194;0;0;2846;463;0;548;72;106;436;457;684;0;35;149;186;120;66;0;0;638;564;0;0;48
192;Farsa de Inês Pereira;Gil